# フロントエンド層

フロントエンド（Front-end）層は、DNNにおいて入力データ（波形やスペクトログラムなど）を処理する最初の層です。

## 1D畳み込み層

その名の通り、1次元方向に畳み込みを行う層です。波形データをスペクトログラムに変換する代わりに、「学習可能（learnable）な」特徴抽出器と見なした畳み込み層に入力することで、よりタスクに適した特徴量を得る、という動機で使われます。

音楽音源分離モデルのフロントエンド層として多用されています[^LM19] [^demucs19]。

[^LM19]: Yi Luo and Nima Mesgarani. Conv-tasnet: surpassing ideal time-frequency magnitude masking for speech separation. IEEE ACM Trans. Audio Speech Lang. Process., 27(8):1256–1266, 2019. URL: https://doi.org/10.1109/TASLP.2019.2915167.

[^demucs19]: Alexandre Défossez, Nicolas Usunier, Léon Bottou, and Francis R. Bach. Demucs: deep extractor for music sources with extra unlabeled data remixed. CoRR, 2019. URL: http://arxiv.org/abs/1909.01174.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio

wav, sr = torchaudio.load("assets/example1.ogg")
wav = wav.unsqueeze(0)
print('入力：', wav.shape)

conv_1d = nn.Conv1d(
    in_channels=2,
    out_channels=64,
    kernel_size=5,
    stride=1,
    padding=2,  # stride=1, padding=(kernel_size - 1) // 2　にすれば、出力サイズが入力サイズと同じになる
    dilation=1,
    groups=1,
)
out = conv_1d(wav)
print('出力：', out.shape)



入力： torch.Size([1, 2, 3120768])
出力： torch.Size([1, 64, 3120768])


スペクトログラムの周波数次元を「チャンネル次元」と見なせば、スペクトログラムを入力とすることも可能です。

## Dilated-Conv-1D 

初めて「サンプル単位の逐次音声合成」という力技に挑戦したWaveNet[^wavenet16]にて提案された畳み込み層です。畳み込みカーネルを「飛ばし飛ばし」に適用することで、受容野（receptive field）をなるべく広げることが目的で、大域的な構造を考慮するようなモデルに使われます。

サンプル単位の音声合成は流石に効率が悪すぎましたが、Dilated-Conv-1D層を重ねたTCN（Temporal Convolutional Network）は音楽解析タスクで多用されています。

[^wavenet16]: Aäron van den Oord et al., Wavenet: A generative model for raw audio. In Alan W. Black, editor, The 9th ISCA Speech Synthesis Workshop, SSW 2016, Sunnyvale, CA, USA, September 13-15, 2016, 125. ISCA, 2016.

In [2]:
dil_conv_1d = nn.Conv1d(
    in_channels=2,
    out_channels=64,
    kernel_size=5,
    stride=1,
    padding="same",     # "same"に設定すれば、入出力長が等しくなるようにpaddingを自動で設定してくれる
    dilation=8,         # dilation>1 だとdilated convolution
    groups=1,
)
out = dil_conv_1d(wav) 
print('出力：', out.shape)

出力： torch.Size([1, 64, 3120768])


## パラメトリックフロントエンド

ニューラルネットのパラメーターは、ベクトルや配列だけでなく、「パラメーター付きの関数」で表すこともできます。
パラメーターの値に制約を付けることで、より解釈しやすい処理になることが期待されています。

その初の試みとして、**SincNet**[^RB18]は畳み込みカーネルを「SinC関数」で表す畳み込み層を提案しました。
SincNetは、畳み込みカーネル：$\omega_n, n\in\{0, ..., N-1\}$を、学習可能なパラメーター$[f_1, f_2]$からなる関数で表します。

$$
\omega_n^{f_1, f_2}=2f_2 sinc(2\pi f_2n)-2f_1 sinc(2\pi f_1n)
$$

$$
sinc(x) = \frac{sin(x)}{x}
$$

このカーネルを用いた畳み込み処理は「カットオフ周波数が$[f_1, f_2]$のバンドパスフィルタ」に相当します。

畳み込み層が話者識別タスクにとって重要な帯域を自動的に学習するため、SincNetは一般的な畳み込みニューラルネットワークよりも高い精度を達成したほか、学習の収束速度も大きく向上したとのことです。


[^RB18]: Mirco Ravanelli and Yoshua Bengio. Speaker recognition from raw waveform with sincnet. In 2018 IEEE Spoken Language Technology Workshop, SLT 2018, Athens, Greece, December 18-21, 2018, 1021–1028. IEEE, 2018. URL: https://doi.org/10.1109/SLT.2018.8639585

In [ ]:
"""
Source code borrowed from: 
https://geoffroypeeters.github.io/deeplearning-101-audiomir_book/bricks_frontend.html

"""

import numpy as np
import matplotlib.pyplot as plt


class SincConv_fast(nn.Module):
    """Sinc-based convolution
    Parameters
    ----------
    in_channels : `int`
        Number of input channels. Must be 1.
    out_channels : `int`
        Number of filters.
    kernel_size : `int`
        Filter length.
    sample_rate : `int`, optional
        Sample rate. Defaults to 16000.
    Usage
    -----
    See `torch.nn.Conv1d`
    Reference
    ---------
    Mirco Ravanelli, Yoshua Bengio,
    "Speaker Recognition from raw waveform with SincNet".
    https://arxiv.org/abs/1808.00158
    """

    @staticmethod
    def to_mel(hz):
        return 2595 * np.log10(1 + hz / 700)

    @staticmethod
    def to_hz(mel):
        return 700 * (10 ** (mel / 2595) - 1)

    def __init__(self, out_channels, kernel_size, sample_rate=16000, in_channels=1,
                 stride=1, padding=0, dilation=1, bias=False, groups=1, min_low_hz=50, min_band_hz=50):

        super(SincConv_fast,self).__init__()

        if in_channels != 1:
            #msg = (f'SincConv only support one input channel '
            #       f'(here, in_channels = {in_channels:d}).')
            msg = "SincConv only support one input channel (here, in_channels = {%i})" % (in_channels)
            raise ValueError(msg)

        self.out_channels = out_channels
        self.kernel_size = kernel_size

        # Forcing the filters to be odd (i.e, perfectly symmetrics)
        if kernel_size%2==0: self.kernel_size=self.kernel_size+1

        self.stride = stride
        self.padding = padding
        self.dilation = dilation

        if bias: raise ValueError('SincConv does not support bias.')
        if groups > 1: raise ValueError('SincConv does not support groups.')

        self.sample_rate = sample_rate
        self.min_low_hz = min_low_hz
        self.min_band_hz = min_band_hz

        # initialize filterbanks such that they are equally spaced in Mel scale
        low_hz = 30
        high_hz = self.sample_rate / 2 - (self.min_low_hz + self.min_band_hz)
        mel_v = np.linspace(self.to_mel(low_hz), self.to_mel(high_hz), self.out_channels + 1)
        hz_v = self.to_hz(mel_v)


        # filter lower frequency (out_channels, 1)
        self.low_hz_v_ = nn.Parameter(torch.Tensor(hz_v[:-1]).view(-1, 1))

        # filter frequency band (out_channels, 1)
        self.band_hz_v_ = nn.Parameter(torch.Tensor(np.diff(hz_v)).view(-1, 1))

        # Hamming window
        #self.window_ = torch.hamming_window(self.kernel_size)
        n_lin = torch.linspace(0, (self.kernel_size/2)-1, steps=int((self.kernel_size/2))) # computing only half of the window
        self.window_ = 0.54-0.46*torch.cos(2*np.pi*n_lin/self.kernel_size);


        # (1, kernel_size/2)
        n = (self.kernel_size - 1) / 2.0
        self.n_ = 2*np.pi*torch.arange(-n, 0).view(1, -1) / self.sample_rate # Due to symmetry, I only need half of the time axes




    def forward(self, waveforms):
        """
        Parameters
        ----------
        waveforms : `torch.Tensor` (batch_size, 1, n_samples)
            Batch of waveforms.
        Returns
        -------
        features : `torch.Tensor` (batch_size, out_channels, n_samples_out)
            Batch of sinc filters activations.
        """

        self.n_ = self.n_.to(waveforms.device)
        self.window_ = self.window_.to(waveforms.device)

        low_v = self.min_low_hz  + torch.abs(self.low_hz_v_)
        high_v = torch.clamp(low_v + self.min_band_hz + torch.abs(self.band_hz_v_),
                           self.min_low_hz,
                           self.sample_rate/2)
        band_v = (high_v - low_v)[:,0]

        f_times_t_low = torch.matmul(low_v, self.n_)
        f_times_t_high = torch.matmul(high_v, self.n_)

        band_pass_left = ((torch.sin(f_times_t_high) - torch.sin(f_times_t_low)) / (self.n_/2)) * self.window_ # Equivalent of Eq.4 of the reference paper (SPEAKER RECOGNITION FROM RAW WAVEFORM WITH SINCNET). I just have expanded the sinc and simplified the terms. This way I avoid several useless computations.
        band_pass_center = 2 * band_v.view(-1,1)
        band_pass_right= torch.flip(band_pass_left, dims=[1])

        band_pass=torch.cat([band_pass_left,
                             band_pass_center,
                             band_pass_right],dim=1)

        band_pass = band_pass / (2*band_v[:,None])

        self.filters = (band_pass).view(self.out_channels, 1, self.kernel_size)

        return F.conv1d(waveforms, self.filters, stride=self.stride, padding=self.padding, dilation=self.dilation, bias=None, groups=1)


model = SincConv_fast(out_channels=80, kernel_size=251, padding=125, sample_rate=16000, in_channels=1)
X = torch.randn(2, 1, 16000)
Y = model(X)
print(Y.shape)


torch.Size([2, 80, 16000])
